In [37]:
import pandas as pd

In [38]:
train_data_path = ("C:/Users/Palash/Desktop/Python Programming course/train.csv")
test_data_path = ("C:/Users/Palash/Desktop/Python Programming course/test.csv")
train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

In [39]:
print (train.shape)
print(test.shape)

(614, 13)
(367, 12)


In [40]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [41]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [44]:
from time import time
from pyelasticsearch import ElasticSearch



In [45]:
CHUNKSIZE=100

index_name_train = "loan_prediction_train"
doc_type_train = "av-lp_train"

index_name_test = "loan_prediction_test"
doc_type_test = "av-lp_test"

In [46]:
def index_data(data_path,chunksize,index_name, doc_type):
    f=open(data_path)
    csvfile=pd.read_csv(f, iterator=True, chunksize=CHUNKSIZE)
    es=ElasticSearch("http://localhost:9200/") #points to the elastic search instance
    try:        
        es.delete_index(index_name)
    except:
        pass
    es.create_index(index_name)
    
    for i,df in enumerate(csvfile):
        records=df.where(pd.notnull(df),None).T.to_dict()
        #print(records)
        list_records=[records[it] for it in records]
        #print(list_records)
        try:
            es.bulk_index(index_name,doc_type,list_records)
        except :
            print("error!, skiping chunk!")
            pass

In [49]:
index_data(train_data_path, CHUNKSIZE, index_name_train, doc_type_train) # Indexing train data
index_data(test_data_path, CHUNKSIZE, index_name_test, doc_type_test) # Indexing test data

DELETE /loan_prediction_train [status:404 request:0.000s]
PUT /loan_prediction_train [status:406 request:0.000s]


ElasticHttpError: (406, 'Content-Type header [] is not supported')